In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import backend as K


In [2]:
# Load the dataset
data = pd.read_csv('./labeled_data.csv')
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [3]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    precisions = precision(y_true, y_pred)
    recalls = recall(y_true, y_pred)
    return 2*((precisions*recalls)/(precisions+recalls+K.epsilon()))

In [4]:
# Extract the text and label columns
text = data['tweet']
labels = data['class']

# Tokenize
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)
vocab_size = len(tokenizer.word_index) + 1

# Padding
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length)
max_sequence_length = max(len(seq) for seq in sequences)

# this for num of labels (harusnya ada 3)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)
one_hot_labels = to_categorical(encoded_labels)

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, one_hot_labels, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_sequence_length))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.5))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',f1,precision, recall])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
620/620 [==============================] - 42s 63ms/step - loss: 0.2843 - accuracy: 0.8422 - f1: 0.8292 - precision: 0.8455 - recall: 0.8173 - val_loss: 0.2100 - val_accuracy: 0.8840 - val_f1: 0.8851 - val_precision: 0.8946 - val_recall: 0.8761
Epoch 2/10
620/620 [==============================] - 39s 63ms/step - loss: 0.1590 - accuracy: 0.9150 - f1: 0.9154 - precision: 0.9224 - recall: 0.9087 - val_loss: 0.2127 - val_accuracy: 0.8868 - val_f1: 0.8879 - val_precision: 0.8934 - val_recall: 0.8826
Epoch 3/10
620/620 [==============================] - 39s 63ms/step - loss: 0.1006 - accuracy: 0.9478 - f1: 0.9479 - precision: 0.9505 - recall: 0.9454 - val_loss: 0.2802 - val_accuracy: 0.8638 - val_f1: 0.8637 - val_precision: 0.8662 - val_recall: 0.8612
Epoch 4/10
620/620 [==============================] - 42s 68ms/step - loss: 0.0626 - accuracy: 0.9700 - f1: 0.9699 - precision: 0.9704 - recall: 0.9693 - val_loss: 0.2925 - val_accuracy: 0.8628 - val_f1: 0.8639 - val_precision: 0.86

In [5]:
import nltk
from nltk.corpus import stopwords
import re

### Preprocessing
it consist of:
- removing unique char (emoji, flags, non-alphabet char)
- url links
- stopwords
- changing @xxx into user
- null tweets

In [6]:
import demoji
import html

In [7]:
stop_words = set(stopwords.words('english'))

def remove_emojis(raw_text):
    text = html.unescape(raw_text)
    text = demoji.replace(text, '')
    return text

def remove_stopwords(raw_text):
    tokenize = nltk.word_tokenize(raw_text)
    text = [word for word in tokenize if not word.lower() in stop_words]
    text = " ".join(text)

    return text
# Preprocessing
def remove_url(raw_text):
    url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    text = re.sub(url_regex, '', raw_text)

    return text

def change_user(raw_text):
    regex = r"@([^ ]+)"
    text = re.sub(regex, "user", raw_text)
    return text
        

In [8]:
test_list = [
    'got ya bitch tip toeing on my hardwood floors "" &#128514; http://t.co/cOU2WQ5L4q"',
    "pussy is a powerful drug @j__nwmv "" &#128517; #HappyHumpDay http://t.co/R8jsymiB5b",
    "...Son of a bitch took my Tic Tacs.",
    '"@2015seniorprobs: I probably wouldn&#8217;t mind school as much if we didn&#8217;t have to deal with bitch ass teachers"". Retweet',
    '"""..All I wanna do is get money and fuck model bitches!"" - Russell Simmons"',
    "@AutoWorId: Hennessey Venom GT &#128584; http://t.co/i8eGMnKaJ9"" that's one sexy bitch"
]

for x in test_list:
    print(x)

got ya bitch tip toeing on my hardwood floors "" &#128514; http://t.co/cOU2WQ5L4q"
pussy is a powerful drug @juanwmv  &#128517; #HappyHumpDay http://t.co/R8jsymiB5b
...Son of a bitch took my Tic Tacs.
"@2015seniorprobs: I probably wouldn&#8217;t mind school as much if we didn&#8217;t have to deal with bitch ass teachers"". Retweet
"""..All I wanna do is get money and fuck model bitches!"" - Russell Simmons"
@AutoWorId: Hennessey Venom GT &#128584; http://t.co/i8eGMnKaJ9 that's one sexy bitch


In [9]:
def remove_noise(datas):
    clean = []
    # change the @xxx into "user"
    clean = [change_user(text) for text in datas]
    # remove emojis (specifically unicode emojis)
    clean = [remove_emojis(text) for text in clean]
    # remove urls
    clean = [remove_url(text) for text in clean]
    # remove stopwords
    # clean = [remove_stopwords(text) for text in clean]
    return clean

test_list = remove_noise(test_list)
for x in test_list:
    print(x)

got ya bitch tip toeing on my hardwood floors ""  "
pussy is a powerful drug user   #HappyHumpDay 
...Son of a bitch took my Tic Tacs.
"user I probably wouldn’t mind school as much if we didn’t have to deal with bitch ass teachers"". Retweet
"""..All I wanna do is get money and fuck model bitches!"" - Russell Simmons"
user Hennessey Venom GT   that's one sexy bitch


In [10]:
for x in stop_words:
    print(x)

other
in
ll
s
o
same
shan't
is
a
herself
ours
yourselves
through
couldn't
wouldn
you're
which
its
was
over
theirs
their
those
doing
shouldn
didn't
she
isn't
so
too
into
once
being
an
isn
they
more
you
until
your
all
hasn't
hers
been
am
his
ma
when
mightn
only
mightn't
did
don't
against
wasn
because
re
about
the
mustn't
very
haven
you'll
then
will
wasn't
that'll
have
ourselves
on
needn
doesn't
mustn
itself
our
themselves
each
didn
yourself
most
him
her
how
who
nor
that
up
whom
m
needn't
it's
my
i
ve
does
has
wouldn't
having
after
shouldn't
at
we
them
if
or
now
hadn
there
why
what
me
are
should've
yours
some
than
haven't
but
for
d
down
by
just
both
with
he
you've
from
won
further
don
couldn
aren't
below
had
between
to
be
hadn't
before
while
out
y
himself
do
off
and
as
under
t
weren
you'd
myself
she's
again
such
of
no
own
ain
won't
this
these
during
above
doesn
weren't
hasn
shan
were
aren
few
any
not
should
here
it
can
where


- got ya bitch tip toeing hardwood floors `` '' ``
- pussy powerful drug user # HappyHumpDay
- ... Son bitch took Tic Tacs .
- `` user probably ’ mind school much ’ deal bitch ass teachers '' '' . Retweet
- `` `` '' .. wan na get money fuck model bitches ! '' '' - Russell Simmons ''
- user Hennessey Venom GT 's one sexy bitch

I Forgot why i put this. only god knows what I'm thinking back then

In [11]:
# Split into training and testing sets
train_text, test_text, train_labels, test_labels = train_test_split(text, labels, test_size=0.2, random_state=42)

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_text)
vocab_size = len(tokenizer.word_index) + 1

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train_text)
test_sequences = tokenizer.texts_to_sequences(test_text)

# Pad sequences to ensure uniform length
max_sequence_length = max(len(seq) for seq in train_sequences)
train_sequences = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post')
test_sequences = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')

# Define the LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_sequence_length))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='softmax'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_sequences, train_labels, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_sequences, test_labels)

# Make predictions
predictions = model.predict(test_sequences)

Epoch 1/10
496/496 [==============================] - 34s 64ms/step - loss: -2.4334 - accuracy: 0.7722 - val_loss: -4.0524 - val_accuracy: 0.7844
Epoch 2/10
496/496 [==============================] - 29s 59ms/step - loss: -5.3477 - accuracy: 0.7722 - val_loss: -6.8819 - val_accuracy: 0.7844
Epoch 3/10
496/496 [==============================] - 28s 56ms/step - loss: -8.1396 - accuracy: 0.7722 - val_loss: -9.7407 - val_accuracy: 0.7844
Epoch 4/10
496/496 [==============================] - 27s 55ms/step - loss: -10.9515 - accuracy: 0.7722 - val_loss: -12.6367 - val_accuracy: 0.7844
Epoch 5/10
496/496 [==============================] - 26s 53ms/step - loss: -13.7553 - accuracy: 0.7722 - val_loss: -15.4700 - val_accuracy: 0.7844
Epoch 6/10
496/496 [==============================] - 26s 52ms/step - loss: -16.5498 - accuracy: 0.7722 - val_loss: -18.3350 - val_accuracy: 0.7844
Epoch 7/10
496/496 [==============================] - 26s 52ms/step - loss: -19.3322 - accuracy: 0.7722 - val_loss: -2